<a href="https://colab.research.google.com/github/Kajcik/Projekty/blob/main/historia_kurs%C3%B3w_walut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Funkcja do przypisywania nazw walut
def nazwa_waluty(kod):
    waluty = {
        'USD': 'Dolar amerykański',
        'EUR': 'Euro',
        'GBP': 'Funt szterling',
        'CHF': 'Frank szwajcarski',
        'PLN': 'Złoty',
        # Można dodać więcej walut
    }
    return waluty.get(kod, 'Nieznana waluta')

# Funkcja do generowania listy dat pomiędzy dwiema datami
def generuj_dat_range(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    delta = end - start
    daty = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(delta.days + 1)]
    return daty

# Funkcja do pobierania danych kursów dla określonej waluty i daty
def pobierz_kurs(waluta, data):
    try:
        link = f"http://api.nbp.pl/api/exchangerates/rates/A/{waluta}/{data}/"
        strona = requests.get(link)
        dane = strona.json()
        kurs = dane['rates'][0]['mid']
        tabela_referencyjna = dane['rates'][0]['no']
        return kurs, tabela_referencyjna
    except:
        return None, None

# Funkcja do pobierania wszystkich kodów walut z API NBP
def pobierz_dostepne_waluty():
    try:
        link = "http://api.nbp.pl/api/exchangerates/tables/A/"
        strona = requests.get(link)
        dane = strona.json()
        waluty = {entry['code']: entry['currency'] for entry in dane[0]['rates']}
        return waluty
    except:
        print("Błąd podczas pobierania dostępnych walut.")
        return {}

print("KALKULATOR WALUT")

# Pobieranie i wyświetlanie dostępnych kodów walut
dostepne_waluty = pobierz_dostepne_waluty()
if dostepne_waluty:
    print("Dostępne waluty i ich kody:")
    for kod, nazwa in dostepne_waluty.items():
        print(f"{kod} - {nazwa}")

# Wprowadzenie listy walut
waluty = input("Podaj kody walut oddzielone przecinkami (np. USD, EUR): ").split(',')

# Wprowadzenie zakresu dat
data_start = input("Podaj datę początkową w formacie rrrr-mm-dd: ")
data_koniec = input("Podaj datę końcową w formacie rrrr-mm-dd: ")

# Generowanie zakresu dat
daty = generuj_dat_range(data_start, data_koniec)

# Lista na wyniki
wyniki = []

# Pobieranie danych dla każdej waluty i daty
for waluta in waluty:
    waluta = waluta.strip().upper()
    nazwa = nazwa_waluty(waluta) if waluta not in dostepne_waluty else dostepne_waluty[waluta]

    for data in daty:
        kurs, tabela_referencyjna = pobierz_kurs(waluta, data)

        if kurs is not None:
            wyniki.append({
                'Waluta': nazwa,
                'Kod waluty': waluta,
                'Data': data,
                'Kurs (PLN)': kurs
            })
        else:
            print(f"Brak danych dla {waluta} na dzień {data}")

# Tworzenie tabeli z wynikami
tabela = pd.DataFrame(wyniki)

# Wyświetlanie tabeli
if not tabela.empty:
    print(tabela)
else:
    print("Brak danych do wyświetlenia.")

KALKULATOR WALUT
Dostępne waluty i ich kody:
THB - bat (Tajlandia)
USD - dolar amerykański
AUD - dolar australijski
HKD - dolar Hongkongu
CAD - dolar kanadyjski
NZD - dolar nowozelandzki
SGD - dolar singapurski
EUR - euro
HUF - forint (Węgry)
CHF - frank szwajcarski
GBP - funt szterling
UAH - hrywna (Ukraina)
JPY - jen (Japonia)
CZK - korona czeska
DKK - korona duńska
ISK - korona islandzka
NOK - korona norweska
SEK - korona szwedzka
RON - lej rumuński
BGN - lew (Bułgaria)
TRY - lira turecka
ILS - nowy izraelski szekel
CLP - peso chilijskie
PHP - peso filipińskie
MXN - peso meksykańskie
ZAR - rand (Republika Południowej Afryki)
BRL - real (Brazylia)
MYR - ringgit (Malezja)
IDR - rupia indonezyjska
INR - rupia indyjska
KRW - won południowokoreański
CNY - yuan renminbi (Chiny)
XDR - SDR (MFW)
Podaj kody walut oddzielone przecinkami (np. USD, EUR): usd, eur
Podaj datę początkową w formacie rrrr-mm-dd: 2024-10-21
Podaj datę końcową w formacie rrrr-mm-dd: 2024-10-30
Brak danych dla USD na d